In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [4]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

23/09/05 23:14:01 WARN Utils: Your hostname, riyaz-Aspire-VX5-591G resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface enp3s0)
23/09/05 23:14:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 23:14:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Reading CSV


In [5]:
#reading csv file
listing_df = spark.read.csv('cleaned_data/clean_listing_csv/', header=True, inferSchema=True)
review_df = spark.read.csv('cleaned_data/clean_reviews_csv/', header=True, inferSchema=True)
calender_df = spark.read.csv('cleaned_data/clean0_calendar_csv/', header=True, inferSchema=True)




In [6]:
listing_df.show(truncate =False)


+--------+-------------------------------------+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
calender_df.show()

+----------+----------+---------+-----+
|listing_id|      date|available|price|
+----------+----------+---------+-----+
|  12147973|2017-09-05|    false| null|
|  12147973|2017-09-04|    false| null|
|  12147973|2017-09-03|    false| null|
|  12147973|2017-09-02|    false| null|
|  12147973|2017-09-01|    false| null|
|  12147973|2017-08-31|    false| null|
|  12147973|2017-08-30|    false| null|
|  12147973|2017-08-29|    false| null|
|  12147973|2017-08-28|    false| null|
|  12147973|2017-08-27|    false| null|
|  12147973|2017-08-26|    false| null|
|  12147973|2017-08-25|    false| null|
|  12147973|2017-08-24|    false| null|
|  12147973|2017-08-23|    false| null|
|  12147973|2017-08-22|    false| null|
|  12147973|2017-08-21|    false| null|
|  12147973|2017-08-20|    false| null|
|  12147973|2017-08-19|    false| null|
|  12147973|2017-08-18|    false| null|
|  12147973|2017-08-17|    false| null|
+----------+----------+---------+-----+
only showing top 20 rows



In [8]:
review_df.show()

+----------+--------+----------+-----------+-------------+--------------------+
|listing_id|      id|      date|reviewer_id|reviewer_name|            comments|
+----------+--------+----------+-----------+-------------+--------------------+
|   1178162| 4724140|2013-05-21|    4298113|      Olivier|My stay at islam'...|
|   1178162| 4869189|2013-05-29|    6452964|    Charlotte|Great location fo...|
|   1178162| 5003196|2013-06-06|    6449554|    Sebastian|We really enjoyed...|
|   1178162| 5150351|2013-06-15|    2215611|       Marine|The room was nice...|
|   1178162| 5171140|2013-06-16|    6848427|       Andrew|Great location. J...|
|   1178162| 5198929|2013-06-17|    6663826|        Arndt|A truely exeption...|
|   1178162| 6702817|2013-08-21|    8099222|      Maurice|It was a really n...|
|   1178162| 6873023|2013-08-28|    7671888|       Elodie|Islam is a very n...|
|   1178162| 7646702|2013-09-28|    8197342|    Arkadiusz|The place is real...|
|   1178162| 8094418|2013-10-15|    9040

**UDF task**

Create a UDF to categorize listings in Boston based on their price range (e.g., budget, mid-range, luxury). Apply this UDF to the listings_df dataframe to categorize the listings.


In [9]:
from pyspark.sql.functions import udf, when, col
from pyspark.sql.types import StringType


# Define the UDF to categorize listings based on their price
def categorize_price_range(price):
    if price < 5000:
        return "Budget"
    elif price >= 5000 and price < 10000:
        return "Mid-Range"
    else:
        return "Luxury"

# Register the UDF with Spark
categorize_price_udf = udf(categorize_price_range, StringType())

# Apply the UDF to categorize listings
listing_df_q1 = listing_df.withColumn("price_range_category", categorize_price_udf(col("price").cast("float")))

# Show the resulting DataFrame with the new category column
listing_df_q1.select("id", "price", "price_range_category").show()




+--------+-----+--------------------+
|      id|price|price_range_category|
+--------+-----+--------------------+
|12147973|25000|              Luxury|
| 3075044| 6500|           Mid-Range|
|    6976| 6500|           Mid-Range|
| 1436513| 7500|           Mid-Range|
| 7651065| 7900|           Mid-Range|
|12386020| 7500|           Mid-Range|
| 5706985|10000|              Luxury|
| 2843445| 7500|           Mid-Range|
|  753446| 5800|           Mid-Range|
|  849408|22900|              Luxury|
| 5434353|14500|              Luxury|
|  225979| 6000|           Mid-Range|
| 3420384|16500|              Luxury|
| 2583074| 4000|              Budget|
|13251243|12000|              Luxury|
|  225834| 7000|           Mid-Range|
| 6400432|15000|              Luxury|
| 5498472|17500|              Luxury|
|  894539| 9500|           Mid-Range|
|  879929| 9000|           Mid-Range|
+--------+-----+--------------------+
only showing top 20 rows




**Simple Operator Task**: 


Combine reviews_df and listings_df to study the number of reviews for each property type in Boston. Order the results by property type.

In [49]:
from pyspark.sql.functions import desc

join_expression = review_df["listing_id"] == listing_df["id"]


#  Join reviews_df with listings_df on "listing_id"
combined_df = review_df.join(listing_df, join_expression, "inner")

# Group by "property_type" and count the number of reviews for each property type
review_count_by_property_type = combined_df.groupBy("property_type") \
    .count() \
    .withColumnRenamed("count", "ReviewCount")

# Order the results by property type
review_count_by_property_type = review_count_by_property_type.orderBy(col("ReviewCount").desc())

# Show the resulting DataFrame
review_count_by_property_type.show()

+---------------+-----------+
|  property_type|ReviewCount|
+---------------+-----------+
|      Apartment|      36826|
|          House|      12438|
|    Condominium|       2799|
|Bed & Breakfast|       1314|
|      Townhouse|        669|
|           Loft|        638|
|           Boat|        187|
|          Other|         55|
|          Villa|         33|
|           null|         28|
|           Dorm|          2|
|     Guesthouse|          1|
+---------------+-----------+



**Window Function Task** 

Identify listings with the highest seasonal price changes by comparing prices in different months.



In [27]:

from pyspark.sql.window import Window
from pyspark.sql import functions as F


# Define a window specification to partition by listing_id and order by month
window_spec = Window.partitionBy("listing_id").orderBy(F.year("date"), F.month("date"))

# Calculate the average price for each listing in each month
avg_price_by_month = calender_df.withColumn("avg_price", F.avg("price").over(window_spec))

# Calculate the price change for each listing in each month compared to the previous month
price_change = avg_price_by_month.withColumn("price_change", F.col("avg_price") - F.lag("avg_price").over(window_spec))

# Find the maximum price change (seasonal price change) for each listing
seasonal_price_change = price_change.groupBy("listing_id").agg(F.max("price_change").alias("max_seasonal_price_change"))

# Rank the listings based on their seasonal price change
ranked_listings = seasonal_price_change.orderBy(F.desc("max_seasonal_price_change"))

# Show the resulting DataFrame with the ranked listings
ranked_listings.show()

+----------+-------------------------+
|listing_id|max_seasonal_price_change|
+----------+-------------------------+
|    447826|                  88650.0|
|  13445221|       25164.705882352944|
|  10004575|                  16000.0|
|   3842286|       14857.142857142855|
|  14813006|        14162.68472906404|
|  11866651|       13598.039215686276|
|   6066455|       13264.705882352937|
|   7633883|       12987.900846338991|
|  13912429|                  12500.0|
|   6326257|       11638.438438438432|
|   8930592|       11444.444444444442|
|   9231486|       11210.909090909088|
|   2881388|       11123.152709359609|
|  13466910|       11114.634146341465|
|   7134081|                  10882.5|
|   8303267|       10838.709677419363|
|  11963384|        10756.36363636364|
|  13306033|       10521.428571428572|
|  14420920|                  10485.0|
|   7377034|       10344.642857142859|
+----------+-------------------------+
only showing top 20 rows



**Simple Operator Task** 

Group listings by room_type and property_type, calculate the average price for each combination, and rename the resulting column as "Average Price." Order the results by the average price in ascending order.



In [30]:
from pyspark.sql.functions import col, avg

# Assuming you have "room_type", "property_type", and "price" columns
listing_df_q = listing_df.withColumnRenamed("room_type", "Room Type")
listing_df_q = listing_df_q.withColumnRenamed("property_type", "Property Type")
listing_df_q = listing_df_q.withColumnRenamed("price", "Price")

# Calculate the average price for each combination of room type and property type
average_price_df = listing_df_q.groupBy("Room Type", "Property Type") \
    .agg(avg("Price").alias("Average Price"))

# Order the results by "Average Price" in ascending order
ordered_average_price_df = average_price_df.orderBy(col("Average Price"))

# Show the resulting DataFrame with the renamed column
ordered_average_price_df.show()


+---------------+---------------+------------------+
|      Room Type|  Property Type|     Average Price|
+---------------+---------------+------------------+
|    Shared room|Bed & Breakfast|            4400.0|
|   Private room|           Dorm|            5000.0|
|   Private room|          Villa|            5500.0|
|Entire home/apt|      Camper/RV|            6000.0|
|    Shared room|           Loft|            6700.0|
|    Shared room|          House|            7450.0|
|   Private room|          House|  8385.42372881356|
|    Shared room|    Condominium|            8900.0|
|   Private room|      Apartment| 9183.187772925765|
|    Shared room|      Apartment| 9654.761904761905|
|   Private room|           Loft|           10240.0|
|   Private room|    Condominium|11075.925925925925|
|   Private room|      Townhouse|11468.965517241379|
|   Private room|           null|           11500.0|
|   Private room|Bed & Breakfast| 13839.39393939394|
|Entire home/apt|Bed & Breakfast|           14

**Pivoting task**

In this question, you'll need to use the pivot operation to transform the data in such a way that the resulting table displays property types as rows and room types as columns. The values in the table should represent the count of Airbnb listings in Boston for each combination of property type and room type.

In [44]:
# Filter the data to include only Boston listings
boston_listings = listing_df.filter(col('city') == 'Boston')

# Group the data by property type and room type to calculate counts
grouped_data = boston_listings.groupBy('property_type', 'room_type') \
    .agg(F.count("*").alias("listing_count"))

# Pivot the data to have property types as rows, room types as columns, and listing counts as values
pivoted_table = grouped_data.groupBy('property_type') \
    .pivot('room_type') \
    .agg(F.sum('listing_count').alias('total_listings')) \
    .fillna(0)

# Show the resulting table
pivoted_table.show()

+---------------+---------------+------------+-----------+
|  property_type|Entire home/apt|Private room|Shared room|
+---------------+---------------+------------+-----------+
|      Apartment|           1273|         423|         38|
|      Townhouse|              8|          27|          0|
|Bed & Breakfast|              2|          28|          1|
|           null|              1|           1|          0|
|           Loft|             23|           2|          1|
|     Guesthouse|              1|           0|          0|
|          Villa|              1|           2|          0|
|   Entire Floor|              2|           0|          0|
|          Other|              0|           9|          3|
|           Dorm|              0|           1|          0|
|    Condominium|            108|          50|          0|
|          House|            106|         259|          4|
|           Boat|             10|           2|          0|
+---------------+---------------+------------+----------